## Setup

In [3]:
import sys
sys.path.append('../../Modules/')
sys.path.append('../../Modules/Processors from Prof')
from Packages import *
from My_Json_processor import *

# Import processors from Prof
from ipynb.fs.full.Utilities import *
from ipynb.fs.full.Json_Processor import *
from ipynb.fs.full.CSV_Processor import *

In [2]:
main_df = ARC_Challenge_Processor_my('ARC-Challenge','ARC-Challenge-Test.jsonl')
main_df = main_df.convert_df()

In [3]:
main_df['full_input'] = main_df['input'] + " " + main_df['choices']

In [4]:
print(main_df.iloc[0]['full_input'])

An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation? A: Planetary density will decrease.
B: Planetary years will become longer.
C: Planetary days will become shorter.
D: Planetary gravity will become stronger.



In [5]:
main_df.head()

,target,input,choices,full_input
0,C,An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?,A: Planetary density will decrease.\nB: Planetary years will become longer.\nC: Planetary days will become shorter.\nD: Planetary gravity will become stronger.\n,An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation? A: Planetary density will decrease.\nB: Planetary years will become longer.\nC: Planetary days will become shorter.\nD: Planetary gravity will become stronger.\n
1,B,A group of engineers wanted to know how different building designs would respond during an earthquake. They made several models of buildings and tested each for its ability to withstand earthquake conditions. Which will most likely result from testing different building designs?,A: buildings will be built faster\nB: buildings will be made safer\nC: building designs will look nicer\nD: building materials will be cheaper\n,A group of engineers wanted to know how different building designs would respond during an earthquake. They made several models of buildings and tested each for its ability to withstand earthquake conditions. Which will most likely result from testing different building designs? A: buildings will be built faster\nB: buildings will be made safer\nC: building designs will look nicer\nD: building materials will be cheaper\n
2,C,The end result in the process of photosynthesis is the production of sugar and oxygen. Which step signals the beginning of photosynthesis?,A: Chemical energy is absorbed through the roots.\nB: Light energy is converted to chemical energy.\nC: Chlorophyll in the leaf captures light energy.\nD: Sunlight is converted into chlorophyll.\n,The end result in the process of photosynthesis is the production of sugar and oxygen. Which step signals the beginning of photosynthesis? A: Chemical energy is absorbed through the roots.\nB: Light energy is converted to chemical energy.\nC: Chlorophyll in the leaf captures light energy.\nD: Sunlight is converted into chlorophyll.\n
3,D,"A physicist wants to determine the speed a car must reach to jump over a ramp. The physicist conducts three trials. In trials two and three, the speed of the car is increased by 20 miles per hour. What is the physicist investigating when he changes the speed?",A: the control\nB: the hypothesis statement\nC: the dependent (responding) variable\nD: the independent (manipulated) variable\n,"A physicist wants to determine the speed a car must reach to jump over a ramp. The physicist conducts three trials. In trials two and three, the speed of the car is increased by 20 miles per hour. What is the physicist investigating when he changes the speed? A: the control\nB: the hypothesis statement\nC: the dependent (responding) variable\nD: the independent (manipulated) variable\n"
4,D,An astronaut drops a 1.0 kg object and a 5.0 kg object on the Moon. Both objects fall a total distance of 2.0 m vertically. Which of the following best describes the objects after they have fallen a distance of 1.0 m?,A: They have each lost kinetic energy.\nB: They have each gained the same amount of potential energy.\nC: They have each lost the same amount of potential energy.\nD: They have each gained one-half of their maximum kinetic energy.\n,An astronaut drops a 1.0 kg object and a 5.0 kg object on the Moon. Both objects fall a total distance of 2.0 m vertically. Which of the following best describes the objects after they have fallen a distance of 1.0 m? A: They have each lost kinetic energy.\nB: They have each gained the same amount of potential energy.\nC: They have each lost the same amount of potential energy.\nD: They have each gained one-half of their maximum kinetic energy.\n


## GROQ API - Testing

In [6]:
import os
from groq import Groq

In [8]:
print(os.environ['GROQ_API_KEY'])

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


In [7]:
client = Groq(
    api_key=os.environ['GROQ_API_KEY'],
)

In [9]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"user",
            "content": main_df.iloc[20]['full_input']
        },
        {
            'role': 'system',
            'content': 'You only answer A, B, C, or D from the multiple choices.'
        }
    ],
    model = "llama3-8b-8192"
)

print(chat_completion.choices[0].message.content)

B: breaking down wastes


## Functions - df_to_csv, GROQ

In [28]:
"""
The function is to generate GROQ answers for the given data.

@params: data(initial dataframe), system_message(message for Gemini system)
@return: sample(dataframe with gemini_output column)
"""

def groq(data, system_message, model_name):

    import time
    total_requests = 0
    successful_requests = 0
    client = Groq(api_key=os.environ['GROQ_API_KEY'],)

    sample = data.copy()
    sample[model_name] = None


    for i in range(len(sample)):
        success = False
        retries = 3

        while not success and retries > 0:
            try:
                total_requests += 1
                
                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role":"user",
                            "content": sample.loc[i, 'full_input']
                        },
                        {
                            'role': 'system',
                            'content': system_message
                        }
                    ],
                    model = model_name
                )

                response = chat_completion.choices[0].message.content

                sample.loc[i, model_name] = response.strip()
                success = True
                successful_requests += 1
                time.sleep(5)

            except Exception as e:
                # print(f"Error: {e}")
                retries -= 1
                time.sleep(5)
                total_requests += 1

    print(f"Total requests made: {total_requests}")
    print(f"Successful requests: {successful_requests}")

    return sample

In [29]:
"""
The function is to export the dataframe to a csv file.

@params: folder_name(name of the folder), df(dataframe)
@return: True if the dataframe is exported successfully, False otherwise
"""

def df_to_csv(folder_name, df, file_name, model_name):
    base_path = r'/Users/ezishr/Library/CloudStorage/OneDrive-UniversityofCincinnati/Undergraduate Research/Check points'
    if df[model_name].isnull().sum() == 0:
        file_path = os.path.join(base_path, folder_name, f'{file_name}.csv')
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        df.to_csv(file_path, index=False)
        print(f"DataFrame exported successfully to {file_path}!")
    else:
        print(f"Export failed: DataFrame contains null values in {model_name}.")
        return False

## My Testing - ARC-Challenge-Dev.jsonl

### Get Dataset

In [1]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

client = Groq(
    api_key=os.environ['GROQ_API_KEY'],
)

In [4]:
arc_challenge = ARC_Challenge_Processor_my('ARC-Challenge','ARC-Challenge-Test.jsonl').convert_df()

In [5]:
arc_challenge['full_input'] = arc_challenge['input'] + " " + arc_challenge['choices']

In [6]:
arc_challenge.drop(columns=['input','choices'],inplace=True)

### GROQ - Model llama-3.3-70b-versatile

In [30]:
llama_33_arc_challenge = groq(arc_challenge, "You can answer only A, B, C, or D to the question","llama-3.3-70b-versatile")

KeyboardInterrupt: 